In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Set parameters
input_shape = (224, 224, 3)  # InceptionV3 input size
batch_size = 32
epochs = 50
num_classes = 6

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Load training data
train_generator = train_datagen.flow_from_directory(
    r"C:\Users\User\OneDrive\Desktop\EmoSense\Face Emotion Recognition\data_AN\train",
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load validation data
val_generator = val_datagen.flow_from_directory(
    r"C:\Users\User\OneDrive\Desktop\EmoSense\Face Emotion Recognition\data_AN\val",
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)




Found 6154 images belonging to 6 classes.
Found 600 images belonging to 6 classes.


In [3]:
# Load InceptionV3 with pre-trained weights
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [4]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    epochs=epochs
)

Epoch 1/50


192/192 [==============================] - 371s 2s/step - loss: 1.8065 - accuracy: 0.3190 - val_loss: 1.6352 - val_accuracy: 0.3420
Epoch 2/50
192/192 [==============================] - 376s 2s/step - loss: 1.5357 - accuracy: 0.3858 - val_loss: 1.6038 - val_accuracy: 0.3507
Epoch 3/50
192/192 [==============================] - 343s 2s/step - loss: 1.4979 - accuracy: 0.4056 - val_loss: 1.5645 - val_accuracy: 0.3524
Epoch 4/50
192/192 [==============================] - 329s 2s/step - loss: 1.4743 - accuracy: 0.4219 - val_loss: 1.5854 - val_accuracy: 0.3715
Epoch 5/50
192/192 [==============================] - 344s 2s/step - loss: 1.4457 - accuracy: 0.4325 - val_loss: 1.5916 - val_accuracy: 0.3733
Epoch 6/50
192/192 [==============================] - 331s 2s/step - loss: 1.4227 - accuracy: 0.4347 - val_loss: 1.5709 - val_accuracy: 0.3663
Epoch 7/50
192/192 [==============================] - 332s 2s/step - loss: 1.4210 - accuracy: 0.4422 - val_loss: 1.5734 - val_accuracy: 0.37

In [5]:
# Unfreeze some of the top layers of the base model and fine-tune
for layer in base_model.layers[-50:]:
    layer.trainable = True

# Recompile with a lower learning rate
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Continue training (fine-tuning)
history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    epochs=50  # Additional epochs for fine-tuning
)

Epoch 1/50
192/192 [==============================] - 362s 2s/step - loss: 1.4156 - accuracy: 0.4410 - val_loss: 1.5962 - val_accuracy: 0.3941
Epoch 2/50
192/192 [==============================] - 346s 2s/step - loss: 1.2993 - accuracy: 0.4966 - val_loss: 1.5802 - val_accuracy: 0.4045
Epoch 3/50
192/192 [==============================] - 346s 2s/step - loss: 1.2361 - accuracy: 0.5178 - val_loss: 1.5644 - val_accuracy: 0.4062
Epoch 4/50
192/192 [==============================] - 346s 2s/step - loss: 1.2085 - accuracy: 0.5314 - val_loss: 1.5844 - val_accuracy: 0.4167
Epoch 5/50
192/192 [==============================] - 347s 2s/step - loss: 1.1677 - accuracy: 0.5511 - val_loss: 1.5924 - val_accuracy: 0.4184
Epoch 6/50
192/192 [==============================] - 344s 2s/step - loss: 1.1383 - accuracy: 0.5580 - val_loss: 1.5932 - val_accuracy: 0.4253
Epoch 7/50
192/192 [==============================] - 339s 2s/step - loss: 1.1112 - accuracy: 0.5696 - val_loss: 1.6149 - val_accuracy: 0.4201

In [6]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.4,
    height_shift_range=0.4,
    shear_range=0.4,
    zoom_range=0.4,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [7]:
# Reduce Learning Rate on Plateau
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=5,
    min_lr=1e-7
)

# Early Stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# Recompile model with potential adjustments
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])


In [8]:
# Continue training (fine-tuning)
history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    epochs=50,  # Additional epochs for fine-tuning
    callbacks=[lr_scheduler, early_stopping]
)

Epoch 1/50
192/192 [==============================] - 281s 1s/step - loss: 0.5822 - accuracy: 0.7888 - val_loss: 1.7514 - val_accuracy: 0.4618 - lr: 1.0000e-05
Epoch 2/50
192/192 [==============================] - 277s 1s/step - loss: 0.5707 - accuracy: 0.7898 - val_loss: 1.8729 - val_accuracy: 0.4427 - lr: 1.0000e-05
Epoch 3/50
192/192 [==============================] - 281s 1s/step - loss: 0.5664 - accuracy: 0.7929 - val_loss: 1.8598 - val_accuracy: 0.4601 - lr: 1.0000e-05
Epoch 4/50
192/192 [==============================] - 280s 1s/step - loss: 0.5660 - accuracy: 0.7950 - val_loss: 1.8886 - val_accuracy: 0.4670 - lr: 1.0000e-05
Epoch 5/50
192/192 [==============================] - 271s 1s/step - loss: 0.5556 - accuracy: 0.7960 - val_loss: 1.8529 - val_accuracy: 0.4531 - lr: 1.0000e-05
Epoch 6/50
192/192 [==============================] - 283s 1s/step - loss: 0.5451 - accuracy: 0.8037 - val_loss: 1.8946 - val_accuracy: 0.4514 - lr: 1.0000e-05
Epoch 7/50
192/192 [====================

In [9]:
# Evaluate the model on test data
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    r"C:\Users\User\OneDrive\Desktop\EmoSense\Face Emotion Recognition\data_AN\test",
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


Found 2400 images belonging to 6 classes.
75/75 [==============================] - 73s 966ms/step - loss: 1.7030 - accuracy: 0.4837
Test Accuracy: 48.37%


In [10]:
# Save the model
model.save('emotion_recognition_inceptionv3.h5')

C:\Users\User\anaconda3\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
